In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch as pt
from tqdm import tqdm

In [5]:
def flatten_index(i,j, col=None):
    if col is None:
        col = N
    return (i-1)*col + (j-1)
    # grid is (N+2,N+2), the A has to be (N^2, N^2)
def unflatten_index(ind, col=None):
    if col is None:
        col = N+2
    i = ind//col
    j = ind%col
    return (i,j)

In [6]:


def create_A(alpha):
    A = np.zeros((N*N, N*N))

    for i in range(1, N+1): # will run 1 to N, so i-1 will be 0 to N-1
        for j in range(1, N+1):

            center = flatten_index(i, j)

            # main diagonal
            A[center, center] = 1 + 4*alpha

            # neighbors
            if i > 1:      # up
                A[center, flatten_index(i-1, j)] = -alpha
            if i < N:      # down
                A[center, flatten_index(i+1, j)] = -alpha
            if j > 1:      # left
                A[center, flatten_index(i, j-1)] = -alpha
            if j < N:      # right
                A[center, flatten_index(i, j+1)] = -alpha

    return A


def impose_boundary_condition(A,b, alpha, left_B = 0, right_B = 0, down_B = 0): #NxN
    
    for row in range(1,N+1):
        for col in range(1,N+1):
            i = row
            j = col
            if row == 1: # neumann
                up = flatten_index(i-1,j)
                center = flatten_index(i,j)
                A[center,center] -= alpha
            if row == N : # dirichlet
                down = flatten_index(i+1,j)
                center = flatten_index(i,j)
                b[center] += alpha * down_B
            if col == 1 :
                left = flatten_index(i,j-1)
                center = flatten_index(i,j)
                b[center] += alpha * left_B
            if col == N:
                right = flatten_index(i,j+1)
                center = flatten_index(i,j)
                b[center] += alpha * right_B
    return A,b

def flatten_grid(grid, remove_padding=True):
    if remove_padding:
        b = grid[1:N+1, 1:N+1].flatten()
    else:
        b = grid.flatten()
    return b

def unflatten_grid(grid, left_B = 0, right_B = 0, down_B = 0, add_padding=True):
    if add_padding:
        interior = grid.reshape((N,N))
        g = np.zeros((N+2, N+2))
        g[1:N+1, 1:N+1] = interior

        g[1:N+1, 0] = left_B
        g[1:N+1, N+1] = right_B
        g[N+1, 1:N+1] = down_B
        g[0, 1:N+1] = g[1,1:N+1]

        g[0,0] = 0
        g[0,N+1] = 0
        g[N+1,0] = 0
        g[N+1, N+1] = 0
    else:
        g = grid.reshape((N+2,N+2))
    return g

def unflatten_all(b, N, left_B=0, right_B=0, down_B=0):
    # b: shape (N*N, K)
    K = b.shape[1]
    interior = b.reshape(N, N, K)

    out = np.zeros((N+2, N+2, K))
    out[1:N+1, 1:N+1, :] = interior

    out[1:N+1, 0, :] = left_B
    out[1:N+1, N+1, :] = right_B
    out[N+1, 1:N+1, :] = down_B
    out[0, 1:N+1, :] = out[1, 1:N+1, :]  # top copies 1st interior row

    return out

def create_padded_grid(N, timesteps, left_B = 0, right_B = 0, down_B = 0):
    g = np.zeros((N+2,N+2,timesteps))
    g[1:N+1, 0] = left_B
    g[1:N+1, N+1] = right_B
    g[N+1, 1:N+1] = down_B

    g[0,0] = 0
    g[0,N+1] = 0
    g[N+1,0] = 0
    g[N+1, N+1] = 0

    return g

In [7]:
def run_exp_NNC(exp_no, sanity_check=False):
    
    left_B = N_i
    right_B = 0
    down_B = N_i
    grid = create_padded_grid(N,timesteps, left_B = left_B, right_B = right_B, down_B = down_B)
    
    b = np.zeros((N**2,timesteps))
    b[:,0] = flatten_grid(grid[:,:,0])
    
    b = pt.tensor(b)
    b_ = b[:,0].clone()
    A = create_A(alpha = alpha_N)
    A_gpu, _ = impose_boundary_condition(A,b_, alpha=alpha_N, left_B = left_B, right_B = right_B, down_B = down_B)
    A_gpu = pt.tensor(A_gpu).to(device)
    for i in tqdm(range(1,timesteps)):
        b_ = b[:,i-1].clone()
        _, b_ = impose_boundary_condition(A,b_, alpha=alpha_N, left_B = left_B, right_B = right_B, down_B = down_B)
        b_ = pt.tensor(b_).to(device)
        x = pt.linalg.solve(A_gpu,b_)
        b[:,i] = x
    
    N_grid = np.zeros_like(grid)
    '''for i, cur_b in tqdm(enumerate(b.T)):
        N_grid[:,:,i] = unflatten_grid(cur_b, left_B = left_B, right_B = right_B, down_B = down_B)
'''
    N_grid = unflatten_all(b,N, left_B = left_B, right_B = right_B, down_B = down_B)
    print('N grid', N_grid[0,0,0])
    left_B = 0
    right_B = C_i
    down_B = 0
    
    grid = create_padded_grid(N,timesteps, left_B = left_B, right_B = right_B, down_B = down_B)
    
    b = np.zeros((N**2,timesteps))
    b[:,0] = flatten_grid(grid[:,:,0])
    
    b = pt.tensor(b)
    b_ = b[:,0].clone()
    A = create_A(alpha = alpha_C)
    A_gpu, _ = impose_boundary_condition(A,b_, alpha=alpha_C, left_B = left_B, right_B = right_B, down_B = down_B)
    A_gpu = pt.tensor(A_gpu).to(device)
    
    for i in tqdm(range(1,timesteps)):
        
        A = create_A(alpha = alpha_N)
        b_ = b[:,i-1].clone()
        _, b_ = impose_boundary_condition(A,b_, alpha=alpha_C,left_B = left_B, right_B = right_B, down_B = down_B)
        b_ = pt.tensor(b_).to(device)
        x = pt.linalg.solve(A_gpu,b_)
        b[:,i] = x
    
    
    C_grid = np.zeros_like(grid)
    '''for i, cur_b in tqdm(enumerate(b.T)):
        C_grid[:,:,i] = unflatten_grid(cur_b, left_B = left_B, right_B = right_B, down_B = down_B)
    '''
    C_grid = unflatten_all(b,N, left_B = left_B, right_B = right_B, down_B = down_B)
    print('C grid', C_grid[0,0,0])
    capacity2 = (N_grid + C_grid).max()
    
    # Move t to first axis if needed
    N_t = np.moveaxis(N_grid, 2, 0)
    C_t = np.moveaxis(C_grid, 2, 0)
    
    V_t = capacity2 - (N_t + C_t)
    V_t = np.clip(V_t, 0, None)
    
    NCV_grid2 = np.stack([N_t, C_t, V_t], axis=-1) / capacity2 # sums to capacity2
    
    N_color = np.array([0,0,1])
    C_color = np.array([1,0,0]) 
    V_color = np.array([0,0,0])
    color_matrix = np.array([N_color, C_color, V_color])
    
    NCV_color1 = np.zeros((timesteps,N+2,N+2,3))
    '''for t in tqdm(range(timesteps)):
        for i in range(N+2):
            for j in range(N+2):
                NCV_color1[t,i,j] = N_color * NCV_grid2[t,i,j,0] + C_color * NCV_grid2[t,i,j,1] + V_color * NCV_grid2[t,i,j,2]
    '''
    NCV_color1 = NCV_grid2 @ color_matrix
    
    exp = {
        'N': N,
        'delta_x':delta_x,
        'delta_t': delta_t,
        'timesteps': timesteps,
        'D_N': D_N,
        'D_C': D_C,
        'C_i':C_i,
        'N_i':N_i,
        'NCV_grid': NCV_grid2,
        'NCV_color':NCV_color1
    }
    
    np.save(f'dataNNC/exp{exp_no}.npy', exp)
    
        

In [8]:
def run_exp_NCN(exp_no):
    left_B = N_i
    right_B = N_i
    down_B = 0
    grid = create_padded_grid(N,timesteps, left_B = left_B, right_B = right_B, down_B = down_B)
    
    b = np.zeros((N**2,timesteps))
    b[:,0] = flatten_grid(grid[:,:,0])
    
    b = pt.tensor(b)
    b_ = b[:,0].clone()
    A = create_A(alpha = alpha_N)
    A_gpu, _ = impose_boundary_condition(A,b_, alpha=alpha_N, left_B = left_B, right_B = right_B, down_B = down_B)
    A_gpu = pt.tensor(A_gpu).to(device)
    
    for i in tqdm(range(1,timesteps)):
        b_ = b[:,i-1].clone()
        _, b_ = impose_boundary_condition(A,b_, alpha=alpha_N, left_B = left_B, right_B = right_B, down_B = down_B)
        b_ = pt.tensor(b_).to(device)
        x = pt.linalg.solve(A_gpu,b_)
        b[:,i] = x
    
    N_grid = np.zeros_like(grid)
    '''for i, cur_b in tqdm(enumerate(b.T)):
        N_grid[:,:,i] = unflatten_grid(cur_b, left_B = left_B, right_B = right_B, down_B = down_B)
'''
    N_grid = unflatten_all(b,N, left_B = left_B, right_B = right_B, down_B = down_B)
    print('N grid', N_grid[0,0,0])
    
    left_B = 0
    right_B = 0
    down_B = C_i
    
    grid = create_padded_grid(N,timesteps, left_B = left_B, right_B = right_B, down_B = down_B)
    
    b = np.zeros((N**2,timesteps))
    b[:,0] = flatten_grid(grid[:,:,0])
    
    b = pt.tensor(b)
    
    b_ = b[:,0].clone()
    A = create_A(alpha = alpha_C)
    A_gpu, _ = impose_boundary_condition(A,b_, alpha=alpha_C, left_B = left_B, right_B = right_B, down_B = down_B)
    A_gpu = pt.tensor(A_gpu).to(device)
    
    for i in tqdm(range(1,timesteps)):
        
        A = create_A(alpha = alpha_N)
        b_ = b[:,i-1].clone()
        _, b_ = impose_boundary_condition(A,b_, alpha=alpha_C,left_B = left_B, right_B = right_B, down_B = down_B)
        b_ = pt.tensor(b_).to(device)
        x = pt.linalg.solve(A_gpu,b_)
        b[:,i] = x
    
    
    C_grid = np.zeros_like(grid)
    '''for i, cur_b in tqdm(enumerate(b.T)):
        C_grid[:,:,i] = unflatten_grid(cur_b, left_B = left_B, right_B = right_B, down_B = down_B)
    '''
    C_grid = unflatten_all(b,N, left_B = left_B, right_B = right_B, down_B = down_B)
    capacity2 = (N_grid + C_grid).max()
    
    # Move t to first axis if needed
    N_t = np.moveaxis(N_grid, 2, 0)
    C_t = np.moveaxis(C_grid, 2, 0)
    
    V_t = capacity2 - (N_t + C_t)
    V_t = np.clip(V_t, 0, None)
    
    NCV_grid2 = np.stack([N_t, C_t, V_t], axis=-1) / capacity2  # sums to capacity2
    
    N_color = np.array([0,0,1])
    C_color = np.array([1,0,0]) 
    V_color = np.array([0,0,0])
    color_matrix = np.array([N_color, C_color, V_color])
    
    NCV_color1 = np.zeros((timesteps,N+2,N+2,3))
    '''for t in tqdm(range(timesteps)):
        for i in range(N+2):
            for j in range(N+2):
                NCV_color1[t,i,j] = N_color * NCV_grid2[t,i,j,0] + C_color * NCV_grid2[t,i,j,1] + V_color * NCV_grid2[t,i,j,2]
    '''
    NCV_color1 = NCV_grid2 @ color_matrix
    
    exp = {
        'N': N,
        'delta_x':delta_x,
        'delta_t': delta_t,
        'timesteps': timesteps,
        'D_N': D_N,
        'D_C': D_C,
        'C_i':C_i,
        'N_i':N_i,
        'NCV_grid': NCV_grid2,
        'NCV_color':NCV_color1
    }
    
    np.save(f'dataNCN/exp{exp_no}.npy', exp)
    
        

In [61]:
#exp 1, C has same diffusivity and concentration
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 5
D_C = 5
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 1
N_i = 1
capacity = C_i + N_i
device='cuda'

In [69]:
alpha_N

2.5

In [62]:
run_exp_NNC(1)
run_exp_NCN(1)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.05it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:31<00:00,  6.61it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:22<00:00,  7.03it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:32<00:00,  6.55it/s]


In [63]:
#exp 2, C has more diffusivity with same concentration
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 5
D_C = 10
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 1
N_i = 1
capacity = C_i + N_i
device='cuda'

In [64]:
run_exp_NNC(2)
run_exp_NCN(2)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.05it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:30<00:00,  6.62it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.06it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:33<00:00,  6.51it/s]


In [65]:
#exp 3, C has more concentration and same diffusivity
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 5
D_C = 5
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 2
N_i = 1
capacity = C_i + N_i
device='cuda'

In [66]:
run_exp_NNC(3)
run_exp_NCN(3)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.05it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:34<00:00,  6.48it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:23<00:00,  6.97it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:34<00:00,  6.45it/s]


In [8]:
#exp 4, C has more concentration and more diffusivity
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 10
D_C = 10
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 2
N_i = 2
capacity = C_i + N_i
device='cuda'

In [9]:
run_exp_NNC(4)
run_exp_NCN(4)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:24<00:00,  6.92it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:29<00:00,  6.67it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.07it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:29<00:00,  6.68it/s]


In [70]:
#exp 5, C has same diffusivity and concentration
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 2
D_C = 2
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 1
N_i = 1
capacity = C_i + N_i
device='cuda'

In [71]:
run_exp_NNC(5)
run_exp_NCN(5)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:22<00:00,  7.01it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:31<00:00,  6.60it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.04it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:37<00:00,  6.33it/s]


In [72]:
#exp 6, C has more diffusivity and same concentration
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 2
D_C = 4
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 1
N_i = 1
capacity = C_i + N_i
device='cuda'

In [73]:
run_exp_NNC(6)
run_exp_NCN(6)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:22<00:00,  7.03it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:34<00:00,  6.45it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.05it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:33<00:00,  6.51it/s]


In [74]:
#exp 7, C has same diffusivity and more concentration
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 2
D_C = 2
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 2
N_i = 1
capacity = C_i + N_i
device='cuda'

In [75]:
run_exp_NNC(7)
run_exp_NCN(7)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:22<00:00,  7.03it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:39<00:00,  6.26it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:22<00:00,  6.99it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_4968\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:38<00:00,  6.30it/s]


In [6]:
#exp 8, C has more diffusivity and more concentration
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 2
D_C = 4
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 2
N_i = 1
capacity = C_i + N_i
device='cuda'

In [7]:
run_exp_NNC(8)
run_exp_NCN(8)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:22<00:00,  7.01it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:29<00:00,  6.66it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:20<00:00,  7.09it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:34<00:00,  6.46it/s]


In [ ]:
# https://www.sciencedirect.com/science/article/pii/S0167577X25012972

# at 200 degree C

"""bcc-Fe
DC = 5.12e-16 m2/s = 5.12 => 1
DN = 1.91e-15 m2/s = 19.1 => 3.73

fcc-Fe
DC = 1.15e-21 m2/s =  11.5 => 11.165
DN = 1.03e-22 m2/s = 1.03 => 1
"""

In [10]:
#exp 10, bcc
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 3.73
D_C = 1
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 1
N_i = 1
capacity = C_i + N_i
device='cuda'

In [11]:
run_exp_NNC(10)
run_exp_NCN(10)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.07it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:31<00:00,  6.62it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.08it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:31<00:00,  6.59it/s]


In [12]:
#exp 11, fcc
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 1
D_C = 11.165
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 1
N_i = 1
capacity = C_i + N_i
device='cuda'

In [13]:
run_exp_NNC(11)
run_exp_NCN(11)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.08it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:38<00:00,  6.30it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:20<00:00,  7.09it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_16176\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:29<00:00,  6.69it/s]


In [6]:
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 2
D_C = 1
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 1
N_i = 1
capacity = C_i + N_i
device='cuda'

In [7]:
run_exp_NNC(9)
run_exp_NCN(9)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_8060\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:22<00:00,  7.02it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_8060\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:31<00:00,  6.57it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.05it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_8060\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:32<00:00,  6.57it/s]


In [ ]:
N = 50
delta_x = 0.1
delta_t = 0.05
timesteps = 1000
D_N = 2
D_C = 1
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 1
N_i = 1
capacity = C_i + N_i
device='cuda'

In [2]:
#exp 20
N = 50
delta_x = 0.1
delta_t = 0.01
timesteps = 1000
D_N = 2
D_C = 2
alpha_N = delta_t * D_N / delta_x
alpha_C = delta_t * D_C / delta_x
C_i = 1
N_i = 1
capacity = C_i + N_i
device='cuda'

In [3]:
alpha_N

0.19999999999999998

In [9]:
run_exp_NNC(9)
run_exp_NCN(9)

  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_15040\1161563782.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:21<00:00,  7.08it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_15040\1161563782.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:30<00:00,  6.62it/s]


C grid 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:20<00:00,  7.10it/s]


N grid 0.0


  0%|                                                                                          | 0/999 [00:00<?, ?it/s]C:\Users\umang\AppData\Local\Temp\ipykernel_15040\3406644134.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b_ = pt.tensor(b_).to(device)
100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [02:29<00:00,  6.67it/s]
